<a href="https://colab.research.google.com/github/rija-ansari/MSE1003H_RijaAnsari/blob/main/Assignment_2/MSE1003_Assignment2_RA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The objective of this assignment is to analyze the relationship between the input design of experiments and the output of remotely collected data from a light sensor. 

The remote data collection was performed using an Opentron 2 (OT-2) at the Acceleration Consortium. The input volumes of red, yellow and blue dyes were prepared by and analyzed by OT-2. 

The goal is to establish a reliable response surface. Conditions for the response surface include:
- Total volume of dyes must be equal to 300 uL
- No dye can have a volume of less than 10 uL
- No dilution

The aim with statistical design of experiments is to utilize a systemized methodology to obtain the maximum amount of informational gain with minimum number of samples.

## Analysis

### Set up environment and assignment folder

Before we begin, let's set up our environment and import the necessary libraries.

If we are using an API key, we need to set uo the environment variable for the API key. 
- Create a .env text file in the root directory of your project and save the key as MPI_KEY=your_api_key_here
- In a Jupyter cell run the following:



In [ ]:
"""
import os
from dotenv import load_dotenv
load_dotenv()
MPI_KEY = os.getenv("MPI_KEY")
"""

Create a virtual environment in the terminal 
- python -m venv .venv  

Create a new text file with the name ".gitignore"
- add the text venv/,pycache/ and .env (if used)

**Issues arrived from multiple python versions that kept conflicting with each other  
Before beginning ensure that 3.13.9 and pymatgen 2025.10.7 are running**


In [ ]:
import sys
import pkg_resources

print("Python version:", sys.version)
print("pymatgen version:", pkg_resources.get_distribution("pymatgen").version)

Check everything is in order:
- make sure this is the main repository on the local drive
    - pwd
- make sure this is the main repository url
    - git remote -v
- we need to add our new files from the assignment folder
    - cd /Users/rija/MSE1003H_RijaAnsari/Assignment_2
    - git add . 
- Move back to the main repo
    - cd .. 
    - git commit -m "Assignment 2 structure update"
    - git pull origin main
    - git push origin main

### Import data

In [ ]:
pip install ternary

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ternary
import plotly.express as px
import colorsys

In [ ]:
cwd = os.getcwd()
print("Current working directory:", cwd)
#open csv file
input = pd.read_csv("colors3.csv")
output = pd.read_csv("color_results.csv")

In [ ]:
volumes = input.copy()
volumes

In [ ]:
#ternary plot 
fig = px.scatter_ternary(
    volumes, 
    a="R", 
    b="Y", 
    c="B",
)

fig.update_traces(marker=dict(color='black', size=5))

fig.update_layout(
    ternary={
        'sum': 100,
        "aaxis": {
            "title": {"text": "Red", "font": {"color": "red", "size": 20}},
            "tickfont": {"color": "red"},
            "linecolor": "red"
        },
        "baxis": {
            "title": {"text": "Yellow", "font": {"color": "yellow", "size": 20}},
            "tickfont": {"color": "black"},
            "linecolor": "yellow"
        },
        "caxis": {
            "title": {"text": "Blue", "font": {"color": "blue", "size": 20}},
            "tickfont": {"color": "blue"},
            "linecolor": "blue"
        }
    }
)
fig.show()

The colour ratios chosen for the 26 data points were distributed so that they would cover even search space throughout the triangle. This was done to ensure that the effects of adding each dye were evenly measured for. There were some limitations on robot volume conditions such that total volume had to be 300 uL and the minimum amount for each dye had to be atleast 10 uL. This deviates from the ideal vertex values of (300,0,0) to determine an "absolute" signal for each dye.

The order of the sample values was also intentional. The experiment proceeds in a manner of descending yellow volume. This is because the Opentron 2 does not rinse between samples and so this order was chosen to lessen the accumulation of the more pigmented dyes (red and blue) in the pipette. 

### 8-channel Output Response

In [ ]:
results = output.copy()
results

In [ ]:
#find the rows with the highest values for each color
yellow_signal = results[results['Yellow'] == 280].index[0]
red_signal = results[results['Red'] == 280].index[0]
blue_signal = results[results['Blue'] == 280].index[0]

red_signal, yellow_signal, blue_signal

In [ ]:
#create a new dataframe with only channel values from last 8 columns
channels = ['ch410', 'ch440', 'ch470', 'ch510', 'ch550', 'ch583', 'ch620', 'ch670']
results_ch = results[channels]

results_ch

In [ ]:
results['ro_raw'] = results['ch620'] + results['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results['yo_raw'] = results['ch510'] + results['ch550'] + results['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results['bo_raw'] = results['ch410'] + results['ch440'] + results['ch470']

# 3. Calculate the total intensity for normalization
results['total_intensity'] = results['ro_raw'] + results['yo_raw'] + results['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results['ro'] = (results['ro_raw'] / results['total_intensity']) * 100
results['yo'] = (results['yo_raw'] / results['total_intensity']) * 100
results['bo'] = (results['bo_raw'] / results['total_intensity']) * 100

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_out = results[['ro', 'yo', 'bo']].copy()

print(results_out)

In [ ]:
#fig = px.scatter_ternary(results_out, a="ro_raw", b="yo_raw", c="bo_raw")
fig = px.scatter_ternary(results_out, a="ro", b="yo", c="bo")
fig.update_layout(title="Ternary Plot of Results")

fig.update_traces(marker=dict(color='black', size=5))

fig.update_layout(
    ternary={
        'sum': 100,
        "aaxis": {
            "title": {"text": "Red", "font": {"color": "red", "size": 20}},
            "tickfont": {"color": "red"},
            "linecolor": "red"
        },
        "baxis": {
            "title": {"text": "Yellow", "font": {"color": "yellow", "size": 20}},
            "tickfont": {"color": "black"},
            "linecolor": "yellow"
        },
        "caxis": {
            "title": {"text": "Blue", "font": {"color": "blue", "size": 20}},
            "tickfont": {"color": "blue"},
            "linecolor": "blue"
        }
    }
)
fig.show()


We are getting a very congested display of our values when the wavelength values are normalized by total intensity. 

Let's see if there's another way of converting our channels. 

Here we are going to compare our max red, yellow and blue values with our middle point in the ternary diagram to see how the sensor responds to an increase in those values. 

This helps us understand how the wavelengths intensity changes relative to our center point.

In [ ]:
yellow1 = results_ch.iloc[yellow_signal] - results_ch.iloc[10]
yellow2 = results_ch.iloc[yellow_signal] - results_ch.iloc[25]
yellow1, yellow2

Here we can see that ch550, ch583 and ch620 show the highest signals for max yellow. 

We also see that the signal is greatly reduced as we move along in our experiment

In [ ]:
red1 = results_ch.iloc[red_signal] - results_ch.iloc[10]
red2 = results_ch.iloc[red_signal] - results_ch.iloc[25]
red1, red2

Red has moderate signals at ch583, and ch620 but definitely not as high as expected. 

Again we see a reduction in signal.

In [ ]:
blue1 = results_ch.iloc[blue_signal] - results_ch.iloc[10]
blue2 = results_ch.iloc[blue_signal] - results_ch.iloc[25]
blue1, blue2

Max blue doesn't seem to show a high signal in the blue wavelengths (ch410, ch440, ch470) but rather is shown more as a decrease of red and yellow wavelengths. 

Even here we a reduction in overall signal.

In [ ]:
results_relative_center1 = results_ch - results_ch.iloc[10]
results_relative_center1

In [ ]:
results_relative_center1['ro_raw'] = results_relative_center1['ch620'] + results_relative_center1['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results_relative_center1['yo_raw'] = results_relative_center1['ch510'] + results_relative_center1['ch550'] + results_relative_center1['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results_relative_center1['bo_raw'] = results_relative_center1['ch410'] + results_relative_center1['ch440'] + results_relative_center1['ch470']

# 3. Calculate the total intensity for normalization
results_relative_center1['total_intensity'] = results_relative_center1['ro_raw'] + results_relative_center1['yo_raw'] + results_relative_center1['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results_relative_center1['ro'] = (results_relative_center1['ro_raw'] / results_relative_center1['total_intensity']) * 100
results_relative_center1['yo'] = (results_relative_center1['yo_raw'] / results_relative_center1['total_intensity']) * 100
results_relative_center1['bo'] = (results_relative_center1['bo_raw'] / results_relative_center1['total_intensity']) * 100

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_center1 = results_relative_center1[['ro', 'yo', 'bo']].copy()

print(results_center1)

In [ ]:
results_center1.iloc[10] = [1, 1, 1]

In [ ]:
#fig = px.scatter_ternary(results_out, a="ro_raw", b="yo_raw", c="bo_raw")
fig = px.scatter_ternary(results_center1, a="ro", b="yo", c="bo")
fig.update_layout(title="Ternary Plot of Results")

fig.update_traces(marker=dict(color='black', size=5))

fig.update_layout(
    ternary={
        'sum': 100,
        "aaxis": {
            "title": {"text": "Red", "font": {"color": "red", "size": 20}},
            "tickfont": {"color": "red"},
            "linecolor": "red"
        },
        "baxis": {
            "title": {"text": "Yellow", "font": {"color": "yellow", "size": 20}},
            "tickfont": {"color": "black"},
            "linecolor": "yellow"
        },
        "caxis": {
            "title": {"text": "Blue", "font": {"color": "blue", "size": 20}},
            "tickfont": {"color": "blue"},
            "linecolor": "blue"
        }
    }
)
fig.show()

In [ ]:
results_relative_center2 = results_ch - results_ch.iloc[25]
results_relative_center2

In [ ]:
results_relative_center2['ro_raw'] = results_relative_center2['ch620'] + results_relative_center2['ch670']

# Yellow: Green-Yellow to Amber wavelengths
results_relative_center2['yo_raw'] = results_relative_center2['ch510'] + results_relative_center2['ch550'] + results_relative_center2['ch583']

# Blue: Violet to Blue-Cyan wavelengths
results_relative_center2['bo_raw'] = results_relative_center2['ch410'] + results_relative_center2['ch440'] + results_relative_center2['ch470']

# 3. Calculate the total intensity for normalization
results_relative_center2['total_intensity'] = results_relative_center2['ro_raw'] + results_relative_center2['yo_raw'] + results_relative_center2['bo_raw']

# 4. Convert to percentages (ro, yo, bo)
# We multiply by 100 so the ternary plot sum equals 100
results_relative_center2['ro'] = (results_relative_center2['ro_raw'] / results_relative_center2['total_intensity']) * 100
results_relative_center2['yo'] = (results_relative_center2['yo_raw'] / results_relative_center2['total_intensity']) * 100
results_relative_center2['bo'] = (results_relative_center2['bo_raw'] / results_relative_center2['total_intensity']) * 100

# 5. Clean up: keep only the outputs you want
#results_out = results[['ro_raw', 'yo_raw', 'bo_raw']].copy()
results_center2 = results_relative_center2[['ro', 'yo', 'bo']].copy()

print(results_center2)

In [ ]:
results_center2.iloc[25] = [1, 1, 1]

In [ ]:
#fig = px.scatter_ternary(results_out, a="ro_raw", b="yo_raw", c="bo_raw")
fig = px.scatter_ternary(results_center2, a="ro", b="yo", c="bo")
fig.update_layout(title="Ternary Plot of Results")
fig.update_traces(marker=dict(color='black', size=5))

fig.update_layout(
    ternary={
        'sum': 100,
        "aaxis": {
            "title": {"text": "Red", "font": {"color": "red", "size": 20}},
            "tickfont": {"color": "red"},
            "linecolor": "red"
        },
        "baxis": {
            "title": {"text": "Yellow", "font": {"color": "yellow", "size": 20}},
            "tickfont": {"color": "black"},
            "linecolor": "yellow"
        },
        "caxis": {
            "title": {"text": "Blue", "font": {"color": "blue", "size": 20}},
            "tickfont": {"color": "blue"},
            "linecolor": "blue"
        }
    }
)
fig.show()

We can see when we compare the results from the first center value that was the 11th sample and the second center value that was the 25th sample, there is a huge variation in the ternary plot. 

This highlights significant issues in precision with our data and its reliability. 

In the first ternary plot, we can also see the skew in the data towards the higher yellow concentration, since it is sampled after the high concentration yellow samples.

In the second plot we a slightly better spread, but does not mimick our response surface at all. 

There are several systemic errors at play. The first is the order of design of experiments with regard to the intensity of yellow recorded. The second is of the system itself. The Opentron sampling system used pipettes that we not disposed of or rinsed in between samples. The light sensor also was not calibrated for varying intensities of each dye. The light sensor was also not sampling in a controlled environment. The overhead lights of the room were on for some of the samples and depending on the activities in the room, this would affect the results. There was also no background correction. There was also no standard procedure for the dye solutions and given that they were uncapped for most of the week, the concentrations of the solutions may vary throughout the duration of the experiment due to sequence in the sampling rotation. 

Random errors include many factors such as temperature and humidity of the room during the experiment. The sampling procedure of the prior experiment would also affect the way the system operated during the experiment (i.e. if it was working properly, needed to be shutdown etc.). Any interactions by the staff scientist with the apparatus during the experiment would also affect the results. 

In [ ]:
#make a copy of results_center1 and make the column names R, Y, B instead of ro, yo, bo
results_center1_copy = results_center1.copy()
results_center1_copy.columns = ['R', 'Y', 'B']
#add a column with student_id with value "OT-2" for all rows
results_center1_copy['student_id'] = 'OT-2'

df_combined = pd.concat([volumes, results_center1_copy], axis=0)


In [ ]:
fig = px.scatter_ternary(
    df_combined, 
    a="R", 
    b="Y", 
    c="B",
    color="student_id",             
   # color_discrete_sequence=['black', 'green'],

)

fig.data[0].marker.color = 'black'  # input
fig.data[1].marker.color = 'green'  # output
fig.update_traces(marker=dict(size=5))

fig.update_layout(
    ternary={
        'sum': 100,
        "aaxis": {
            "title": {"text": "Red", "font": {"color": "red", "size": 20}},
            "tickfont": {"color": "red"},
            "linecolor": "red"
        },
        "baxis": {
            "title": {"text": "Yellow", "font": {"color": "yellow", "size": 20}},
            "tickfont": {"color": "black"},
            "linecolor": "yellow"
        },
        "caxis": {
            "title": {"text": "Blue", "font": {"color": "blue", "size": 20}},
            "tickfont": {"color": "blue"},
            "linecolor": "blue"
        }
    }
)
fig.show()

Above is the comparison plot of the input colour distributions and the and the resulting output from the light sensor with comparison to the first center output (100,100,100) from OT-2. 

We see that there isn't a strong relationship between our input and output values, signifying that this analysis approach is likely not the best way to analyze this data. 

In [ ]:
import scipy.stats as stats
channels = ['ch410', 'ch440', 'ch470', 'ch510', 'ch550', 'ch583', 'ch620', 'ch670']

center_df = results[(results['Red'] == 100) & (results['Yellow'] == 100) & (results['Blue'] == 100)]

all_residuals = []
for ch in channels:
    obs_values = center_df[ch].values
    mean_val = np.mean(obs_values)
    residuals = obs_values - mean_val
    all_residuals.extend(residuals)

#Create the Normal Probability Plot
plt.figure(figsize=(8, 6))
stats.probplot(all_residuals, dist="norm", plot=plt)

plt.title('Normal Probability Plot of Residuals\n(Center Points [100,100,100] across 8 Channels)')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Residuals (Sensor Noise)')
plt.grid(True, linestyle='--', alpha=0.7)

When we compare the two center values we see an a distinct difference in the residuals.

The  S-curve suggests that the residuals display non-normal behaviour, meaning that the error in measurement at the centers is may not be  random Gaussian noise. 

This is likely due to some wavelength channels being more sensitive than others - which is shown in the comparison to the center differences above. 

Let's perform our ANOVA analysis to see if there is a resounding relationship between our input DOE and output 8-channel wavelengths. If there is a relationship this data can be further explored, with learning methods, to come to a better relationship.

### ANOVA Analysis

Analysis of Variance, or ANOVA, is a statistical method used to determine if there are significant differences between the means of three or more independent groups. It works by breaking down the total variation found in a dataset into two components: variation attributable to the factors you are testing (between-group) and variation caused by random noise or error (within-group).

In this exploration, a Three-Factor Factorial ANOVA allows us to see if the individual volumes of Red, Yellow, and Blue dyes affect the sensor intensity (Main Effects) but also if the dyes interact in a way that changes the sensor’s response non-linearly (Interaction Effects).

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

results['Total_Intensity'] = results[channels].sum(axis=1)

# 3. Fit the Three-Factor Factorial Model
# The formula 'Red * Yellow * Blue' includes:
# - Main Effects: Red, Yellow, Blue
# - 2-Way Interactions: Red:Yellow, Red:Blue, Yellow:Blue
# - 3-Way Interaction: Red:Yellow:Blue
model = ols('Total_Intensity ~ Red * Yellow * Blue', data=results).fit()

# 4. Perform ANOVA (Type II Sum of Squares is standard for factorial designs)
anova_table = sm.stats.anova_lm(model, typ=2)

# 5. Display the results
print("--- ANOVA Table for Total Sensor Intensity ---")
print(anova_table)

# 6. (Optional) Interpretation of Coefficients
print("\n--- Model Coefficients (Effect Sizes) ---")
print(model.params)

From the ANOVA table above, we can see that:

- Main Factor Effects: All three primary dyes (Red, Yellow, and Blue) have highly significant main effects ($P < 0.001$).

- Intensity Contribution: The positive coefficients indicate that increasing the volume of any dye significantly increases the total raw signal detected by the sensor.

- Interaction Effects: The p-values greater than 0.05 for the two and three way interaction mixes mean that there is no significant interaction between the dyes. The sensor is able to interpret the dyes as a sum of its parts.

- Dominance: The sensor is the most sensitive to yellow (11.7), red (11.4) and then blue (10.8) as shown in the model coefficients. This is also shown in the descending F values. This means that one unit of yellow dye impacts the overall system response the most. 

- Residuals: The residual sum squares is relatively small to the main factor sum square range. This means that the model is able to explain over 98% of the variance in the data. This also means that experimental precision is high and there is limited impact from pipetting and sensor readings. 

### Summary

This report evaluates an 8-channel optical sensor’s performance using a mixture design of experiments (DOE) to characterize the color space. To ensure full coverage, a simplex-centroid approach was used, varying Red, Yellow, and Blue dye volumes while maintaining a constant total volume.

The sensor's precision was assessed by analyzing the 8-channel output relative to the center values (100, 100, 100). A comparison of the design space between input (volumes) and output (intensities) revealed an inadequate overlap in certain regions. Specifically, the sensor showed slightly compressed resolution compared to the physical input. 

The normal probability plot showed a residual S-curve, suggesting that while the noise is generally controlled, the sensor exhibits non-linear sensitivity or "heavy-tailed" error at the center point.

 However, the ANOVA analysis confirms the experiment's overall validity, showing that Red, Yellow, and Blue are all highly significant main factors (p < 0.001) with negligible interaction effects. Yellow dye producing the strongest marginal increase in signal. This indicates that the sensor responds to the dyes in a predictable, additive manner despite the slight non-linearity identified in the residuals.

This suggests that a machine learning pathway approach is likely the path forward to consolidate the relationship between the input volumes and the 8-channel wavelength outputs. 